In [2]:
import numpy as np
import pandas as pd
import Levenshtein as lev
from itertools import product

In [3]:
sevMil = pd.read_csv("companies_sorted.csv")
ft = pd.read_csv("Forbes Top2000 2017.csv")
db = pd.read_csv("sparql_2022-10-02_13-24-07Z.csv")

In [4]:
sevMil = sevMil[['name']]
sevMil['id'] =  range(len(sevMil))
sevMil['id'] = "7.1M_" + sevMil['id'].astype(str)

In [5]:
ft = ft[['Company']]
ft['id'] =  range(len(ft))
ft['id'] = "ft_" + ft['id'].astype(str)

In [6]:
dbpedia = db[['name']]
dbpedia['id'] =  range(len(db))
dbpedia['id'] = "db_" + dbpedia['id'].astype(str)

C:\Users\seewe\AppData\Local\Temp\ipykernel_22080\118899998.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia['id'] =  range(len(db))
C:\Users\seewe\AppData\Local\Temp\ipykernel_22080\118899998.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia['id'] = "db_" + dbpedia['id'].astype(str)


In [7]:
new_df = pd.DataFrame(product(ft['Company'], dbpedia['name']), columns=["ft","dbpedia"])

new_df["LevScore"] = new_df.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

In [8]:
new_df2 = new_df[new_df['LevScore'] > 0.8]

In [9]:
confirmed_true = new_df2[new_df2['LevScore'] == 1]
edge_cases = new_df2[new_df2['LevScore'] != 1]

In [10]:
true_sample = confirmed_true.sample(200)
false_sample = edge_cases.sample(200)

In [11]:
gold_standard_ft_db = pd.concat([true_sample,false_sample])
gold_standard_ft_db = gold_standard_ft_db.merge(ft, how='left', left_on='ft', right_on='Company')
gold_standard_ft_db.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db.merge(dbpedia, how = 'left', left_on = 'dbpedia', right_on = 'name')
gold_standard_ft_db.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db[['ft_id', 'db_id', 'LevScore', 'ft', 'dbpedia']]
gold_standard_ft_db['label'] = gold_standard_ft_db.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ft_db

,ft_id,db_id,LevScore,ft,dbpedia,label
0,ft_42,db_2604,1.000000,IBM,IBM,True
1,ft_1268,db_6831,1.000000,Jyske Bank,Jyske Bank,True
2,ft_455,db_5001,1.000000,Steinhoff International,Steinhoff International,True
3,ft_1591,db_4623,1.000000,Avis Budget Group,Avis Budget Group,True
4,ft_131,db_984,1.000000,Banco do Brasil,Banco do Brasil,True
...,...,...,...,...,...,...
402,ft_898,db_4964,0.903226,L3 Technologies,RW3 Technologies,False
403,ft_936,db_626,0.900000,CMS Energy,CPS Energy,False
404,ft_1632,db_5148,0.833333,SBI Holdings,CIC Holdings,False
405,ft_481,db_8776,0.864865,Micron Technology,Microchip Technology,False
